In [1]:
import matplotlib.pyplot as plt
import numpy as np
import gvar as gv
import pprint

In [2]:
import re
def parse_data(input_data):
    data = {}

    # Define the order of the baryons
    baryon_order = ["N_E0", "Xi_EO", "Sigma_E0","Lambda_E0", "Xi*_E0", "Delta_E0", "Sigma*_E0"]

    # Define a mapping from your original baryon names to the new ones
    baryon_mapping = {'lambda_z': 'Lambda_E0',
                      'proton': 'N_E0',
                      'sigma_p': 'Sigma_E0',
                      'sigma_star_p': 'Sigma*_E0',
                      'xi_star_z': 'Xi*_E0',
                      'xi_z': 'Xi_EO',
                      'delta_pp':'Delta_E0'}

    for key, value in input_data.items():
        ensemble_name, baryon = key

        best = value['best']
        avg = value['avg']
        
        number_best, uncertainty_best = map(float, re.match(r'(\d+\.\d+)\((\d+)\)', best).groups())
        number_avg, uncertainty_avg = map(float, re.match(r'(\d+\.\d+)\((\d+)\)', avg).groups())

        if ensemble_name not in data:
            data[ensemble_name] = {'nolan_best':{},'nolan_avg':{}}

        # Map the original baryon name to the new one
        new_baryon_name = baryon_mapping[baryon]
        
        # data[ensemble_name][new_baryon_name] = {}
        data[ensemble_name]['nolan_best'][new_baryon_name] = (number_best, uncertainty_best)
        data[ensemble_name]['nolan_avg'] [new_baryon_name] = (number_avg, uncertainty_avg)
        
    for ensemble_name in data:
        for key in ['nolan_best','nolan_avg']:
            data[ensemble_name][key] = {baryon: data[ensemble_name][key].get(
                baryon, (None, None)) for baryon in baryon_order}

    return data



In [3]:
nolan = {}
nolan = gv.load('nolan.p')
nolan_ = parse_data(nolan)  

In [4]:
import re  
import pprint

def parse_latex_table(file_name):
    with open(file_name, "r") as f:
        content = f.read()

    tables = re.split(r'\\end{tabular}', content)[:-1]

    data = {}
    for table in tables:
        lines = table.strip().split("\n")
        if len(lines) < 4:  # If there are not enough lines, skip this table
            continue
        ensemble = lines[2].split("&")[0].strip().replace('$_','_').replace('$', '').replace('{', '').replace('}', '')
        
        data[ensemble] = {}

        for line in lines[3:-1]:  # Skip the first two lines (headers) and the last line (end of the table)
            cols = line.split("&")

            person = cols[0].strip()
            results = [tuple(map(float, re.findall(r"([\d.]+)", item))) if re.findall(r"([\d.]+)", item) else (None, None) for item in cols[1:]]
            results_dict = dict(zip(["N_E0", "Xi_EO", "Sigma_E0", "Lambda_E0", "Xi*_E0", "Delta_E0", "Sigma*_E0"], results))
            # Sorting the person's data and storing it as a list of tuples
            sorted_results = sorted(results_dict.items(), key=lambda item: item[1][0] if item[1][0] is not None else float('inf'))
            data[ensemble][person] = sorted_results

    return data

file_name = "tables.txt"
baryon_order = ["N_E0", "Xi_EO", "Sigma_E0","Lambda_E0", "Xi*_E0", "Delta_E0", "Sigma*_E0"]
data = parse_latex_table(file_name)
for ensemble_name in data:
# Sorting Nolan's data and storing it as a list of tuples
    data[ensemble_name]['nolan_best'] = sorted(nolan_[ensemble_name]['nolan_best'].items(), key=lambda item: item[1][0] if item[1][0] is not None else float('inf'))
    data[ensemble_name]['nolan_avg'] = sorted(nolan_[ensemble_name]['nolan_avg'].items(), key=lambda item: item[1][0] if item[1][0] is not None else float('inf'))

pprint.pprint(data)

{'a06m220L': {'Aaron': [('N_E0', (0.2969, 18.0)),
                        ('Lambda_E0', (0.336, 11.0)),
                        ('Sigma_E0', (0.356, 12.0)),
                        ('Xi_EO', (0.38497, 69.0)),
                        ('Delta_E0', (0.3889, 96.0)),
                        ('Sigma*_E0', (0.4222, 53.0)),
                        ('Xi*_E0', (0.4527, 33.0))],
              'Amy': [('N_E0', (0.2925, 41.0)),
                      ('Lambda_E0', (0.3335, 19.0)),
                      ('Sigma_E0', (0.3538, 22.0)),
                      ('Xi_EO', (0.38361, 98.0)),
                      ('Delta_E0', (0.3943, 90.0)),
                      ('Sigma*_E0', (0.4239, 54.0)),
                      ('Xi*_E0', (0.4541, 30.0))],
              'Daniel': [('N_E0', (0.2986, 19.0)),
                         ('Lambda_E0', (0.3373, 13.0)),
                         ('Sigma_E0', (0.3562, 14.0)),
                         ('Xi_EO', (0.38548, 91.0)),
                         ('Delta_E0', (0.4019, 37.0)),


In [5]:

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def plot_ensemble(ensemble_data, ensemble_name, pdf):
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_title(ensemble_name)

    offset = 0.08
    all_baryon_keys = []
    positions = {}

    for i, (person, results) in enumerate(ensemble_data.items()):
        if person == '\\hline' or person == 'Jinchen':
            continue
        valid_results = {k: v for k, v in results if len(v) > 1 and v[1] is not None}

        x = np.arange(len(valid_results))
        y = [result[1][0] for result in valid_results.items()]
        yerr = [result[1][1] * 0.0001 if result[1][1] is not None else 0 for result in valid_results.items()]

        ax.errorbar(x + i * offset, y, yerr=yerr, fmt="o", capsize=5, label=person)

        # Store the positions of each baryon for this person
        for j, key in enumerate(valid_results.keys()):
            if key in positions:
                positions[key].append(j + i * offset)
            else:
                positions[key] = [j + i * offset]

        all_baryon_keys += [k for k in valid_results.keys() if k not in all_baryon_keys]

    # Calculate the mean position for each baryon
    mean_positions = [np.mean(positions[key]) for key in all_baryon_keys]

    ax.set_xticks(mean_positions)
    ax.set_xticklabels(all_baryon_keys, rotation=45)
    ax.legend()
    pdf.savefig(fig, bbox_inches="tight")
    plt.close(fig)

with PdfPages("output_plots.pdf") as pdf:
    for ensemble_name, ensemble_data in data.items():
        plot_ensemble(ensemble_data, ensemble_name, pdf)
